## Deep Learning with ResNet-50. Image Classification, Binary. Pre-Trained Model Loaded.   

#### Links to GitHub source:  
https://jkjung-avt.github.io/keras-tutorial/  
https://github.com/jkjung-avt/keras-cats-dogs-tutorial/blob/master/train_resnet50.py  
https://github.com/jkjung-avt/keras-cats-dogs-tutorial/blob/master/predict_resnet50.py  

In [1]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.optimizers import Adam
#from preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
cd ~/Downloads/

C:\Users\DJ\Downloads


In [3]:
DATASET_PATH  = './catsdogs/sample'
IMAGE_SIZE    = (224, 224)
NUM_CLASSES   = 2
BATCH_SIZE    = 8  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
#NUM_EPOCHS    = 20
NUM_EPOCHS    = 5
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 3002 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')

****************
Class #0 = cats
Class #1 = dogs
****************


In [6]:
# Build our classifier model based on pre-trained ResNet50:
# 1. we don't include the top (fully connected) layers of ResNet50
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
print(net_final.summary())

C:\Users\DJ\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [7]:
# train the model
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# save trained weights
net_final.save(WEIGHTS_FINAL)

Epoch 1/5
375/375 [==============================] - 2717s 7s/step - loss: 0.7719 - accuracy: 0.7635 - val_loss: 0.0473 - val_accuracy: 0.9570
Epoch 2/5
375/375 [==============================] - 2673s 7s/step - loss: 0.3426 - accuracy: 0.8911 - val_loss: 0.0078 - val_accuracy: 0.9740
Epoch 3/5
375/375 [==============================] - 34520s 92s/step - loss: 0.2678 - accuracy: 0.9265 - val_loss: 0.0031 - val_accuracy: 0.9750
Epoch 4/5
375/375 [==============================] - 2696s 7s/step - loss: 0.2144 - accuracy: 0.9385 - val_loss: 2.5707e-04 - val_accuracy: 0.9790
Epoch 5/5
375/375 [==============================] - 2606s 7s/step - loss: 0.1687 - accuracy: 0.9506 - val_loss: 1.7701e-04 - val_accuracy: 0.9830


In [8]:
#break

In [10]:
import os
import sys
import glob
import argparse

In [11]:
def parse_args():
    """Parse input arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument('path')
    args = parser.parse_args()
    return args

def get_files(path):
    if os.path.isdir(path):
        files = glob.glob(os.path.join(path, '*'))
    elif path.find('*') > 0:
        files = glob.glob(path)
    else:
        files = [path]

    files = [f for f in files if f.endswith('JPG') or f.endswith('jpg')]

    if not len(files):
        sys.exit('No images found by the given path!')

    return files

In [12]:
# !!! LOOKS LIKE PATH ISSUE, JUST PASS IN MANUALLY ?
args = parse_args()
files = get_files(args.path)
cls_list = ['cats', 'dogs']

# load the trained model
net = load_model('model-resnet50-final.h5')

usage: ipykernel_launcher.py [-h] path
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [ ]:
# loop through all files and make predictions
for f in files:
    img = image.load_img(f, target_size=(224,224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], cls_list[i]))